<a href="https://colab.research.google.com/github/evinracher/3008410-intelligent-systems/blob/main/week2/workshop/Taller_LangGraph_Completo_Gemini.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Taller: LangGraph para Chatbots Inteligentes (Gemini)
**Objetivo:** practicar desde lo básico hasta memoria y checkpointing (incluye time travel y streaming).  
**Requisitos:** tener `GOOGLE_API_KEY` en Colab Secrets.

Estructura del taller:
1) Tema + ejemplo corto  
2) “Tu turno”: modifica/crea tu propio código  
3) Preguntas teóricas y “¿qué pasa si…?”

In [ ]:
%pip install -U langgraph langchain-groq pydantic

In [ ]:

import os
from google.colab import userdata

if not os.getenv("GROQ_API_KEY"):
    try:
        os.environ["GROQ_API_KEY"] = userdata.get("GROQ_API_KEY")
    except Exception:
        pass

print("GROQ_API_KEY:", "✅" if os.getenv("GROQ_API_KEY") else "⚠️")

In [ ]:

from langchain_groq import ChatGroq
llm = ChatGroq(model="llama-3.1-8b-instant", temperature=0.2)
print("✅ LLM listo:", llm.model_name)


## 1) Estados, Nodos y Aristas (los 3 pilares)

### Ejemplo 1: grafo lineal (START → nodo → END)

In [ ]:
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END

class S1(TypedDict):
    text: str
    upper: str

def to_upper(state: S1):
    return {"upper": state["text"].upper()}

g = StateGraph(S1)
g.add_node("to_upper", to_upper)
g.add_edge(START, "to_upper")
g.add_edge("to_upper", END)
app1 = g.compile()

app1.invoke({"text": "hola langgraph"})

### Tu turno
1) Crea un nodo `to_lower` que convierta a minúsculas.  
2) Haz un grafo que primero haga `to_upper` y luego `to_lower` (para ver el flujo).

### Preguntas (¿qué pasa si...?)
- ¿Qué pasa si tu estado no tiene la key que el nodo espera?
- ¿Qué pasa si olvidas conectar un nodo a `END`?

## 2) Aristas condicionales (routing)

### Ejemplo 2: decidir ruta según el input

In [ ]:
from typing import Literal

class S2(TypedDict):
    text: str
    kind: str
    out: str

def classify(state: S2):
    t = state["text"].lower()
    kind = "pregunta" if "?" in t else "afirmacion"
    return {"kind": kind}

def responder_pregunta(state: S2):
    resp = llm.invoke(f"Responde breve: {state['text']}")
    return {"out": resp.content}

def resumir_afirmacion(state: S2):
    resp = llm.invoke(f"Resume en 1 frase: {state['text']}")
    return {"out": resp.content}

def route(state: S2) -> Literal["responder_pregunta", "resumir_afirmacion"]:
    return "responder_pregunta" if state["kind"] == "pregunta" else "resumir_afirmacion"

g2 = StateGraph(S2)
g2.add_node("classify", classify)
g2.add_node("responder_pregunta", responder_pregunta)
g2.add_node("resumir_afirmacion", resumir_afirmacion)

g2.add_edge(START, "classify")
g2.add_conditional_edges("classify", route)
g2.add_edge("responder_pregunta", END)
g2.add_edge("resumir_afirmacion", END)

app2 = g2.compile()

print(app2.invoke({"text":"¿Qué es LangGraph?"})["out"])
print(app2.invoke({"text":"LangGraph sirve para orquestar flujos con grafos."})["out"])

### Tu turno
Cambia la regla de routing:
- Si el texto contiene “pasos” → usa un nodo `dar_pasos` (lista 1-3 items).

### Preguntas
- ¿Qué pasa si tu función `route` devuelve un nombre de nodo que no existe?
- ¿Qué pasa si ambos caminos llevan a más nodos y no solo a END?

## 3) Gestión de conversación: historial, truncamiento y resumen

### Ejemplo 3: MessagesState + add_messages

In [ ]:
from langgraph.graph import MessagesState, add_messages
from langchain_core.messages import HumanMessage, AIMessage

# MessagesState ya trae la key "messages" con reducer append-only
def chatbot_node(state: MessagesState):
    resp = llm.invoke(state["messages"])
    return {"messages": [resp]}

g3 = StateGraph(MessagesState)
g3.add_node("chat", chatbot_node)
g3.add_edge(START, "chat")
g3.add_edge("chat", END)
app3 = g3.compile()

state0 = {"messages": [HumanMessage(content="Hola, ¿quién eres?")]}
out = app3.invoke(state0)
out["messages"][-1].content

### Tu turno
- Haz 2 turnos: agrega el mensaje del usuario y vuelve a invocar.
- Observa cómo crece la lista `messages`.
- Implementa una paso para quitar mensajes
- Implementa una paso para resumir mensajes

### Preguntas
- ¿Por qué guardar todo el historial puede ser costoso?
- ¿Qué ventajas tiene “resumir” vs “borrar” mensajes?